In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/PLD_3_Classes_256/Training'
valid_path = '/content/drive/MyDrive/PLD_3_Classes_256/Validation'
test_path = '/content/drive/MyDrive/PLD_3_Classes_256/Testing'

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(inception.output)

In [ ]:
folders = glob('/content/drive/MyDrive/PLD_3_Classes_256/Training/*')

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3281 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 405 images belonging to 3 classes.


In [ ]:
validation_set = validation_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 426 images belonging to 3 classes.


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
)

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 529s 5s/step - accuracy: 0.8824 - loss: 0.5102 - val_accuracy: 0.9086 - val_loss: 0.4486
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 527s 5s/step - accuracy: 0.8905 - loss: 0.5192 - val_accuracy: 0.8840 - val_loss: 0.5480
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 536s 5s/step - accuracy: 0.8999 - loss: 0.4743 - val_accuracy: 0.8963 - val_loss: 0.7016
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 539s 5s/step - accuracy: 0.9079 - loss: 0.5107 - val_accuracy: 0.9185 - val_loss: 0.4078
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 555s 5s/step - accuracy: 0.9335 - loss: 0.3635 - val_accuracy: 0.8988 - val_loss: 0.4852
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 528s 5s/step - accuracy: 0.9416 - loss: 0.2986 - val_accuracy: 0.8889 - val_loss: 0.8716
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 559s 5s/step - accuracy: 0.9220 - loss: 0.4227 - val_accuracy: 0.8790 - val_loss: 1.0496
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 533s 5s/step - accuracy: 0.9343 - loss: 0.4170 - val_accu

In [ ]:
model.save('/content/drive/MyDrive/deepspud.h5')

In [ ]:
model.save('deepspud.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/deepspud.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
test_path='/content/potato_late-blight_08_zoom-Photo-OMAFRA-900x580.jpeg'
img = image.load_img(test_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
predictions = model.predict(img_data)
predicted_classes = predictions.argmax(axis=-1)
classes = ['Early Blight', 'Healthy', 'Late Blight']
predicted_labels = [classes[i] for i in predicted_classes]
print(predictions)
print(predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
[[5.1424781e-15 3.5017125e-11 1.0000000e+00]]
['Late Blight']
